In [ ]:
import tensorflow as tf
import numpy as np
import os

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
text = open('shahnameh.txt' , 'r').read()

In [ ]:
text = text.replace('|' , '')
print(f'Length of the dataset: {len(text)} characters')
print(text[500:1000])

Length of the dataset: 2554631 characters
کی گنجد اوی
بدین آلت رای و جان و زبان
ستود آفریننده را کی توان
به هستیش باید که خستو شوی
ز گفتار بی‌کار یکسو شوی
پرستنده باشی و جوینده راه
به ژرفی به فرمانش کردن نگاه
توانا بود هر که دانا بود
ز دانش دل پیر برنا بود
از این پرده برتر سخن‌گاه نیست
ز هستی مر اندیشه را راه نیست
کنون ای خردمند وصف خرد
بدین جایگه گفتن اندرخورد
کنون تا چه داری بیار از خرد
که گوش نیوشنده زو برخورد
خرد بهتر از هر چه ایزد بداد
ستایش خرد را به از راه داد
خرد رهنمای و خرد دلگشای
خرد دست گیرد به هر دو سرای
ازو شادمانی وزویت غ


In [ ]:
unique_chars = sorted(set(text))
char_to_ind={char :index for index , char in enumerate(unique_chars)}
ind_to_char={index :char for index , char in enumerate(unique_chars)}

vocab_size = len(unique_chars)
print(f"unique_chars:{vocab_size}")

unique_chars:47


In [ ]:
seq_length = 100

input_sequences=[]
target_sequences=[]

for i in range(len(text) -seq_length ):
  input_seq = text[i:i +seq_length]
  target_char = text[i +seq_length]
  input_sequences.append([char_to_ind[char] for char in input_seq])
  target_sequences.append(char_to_ind[target_char])
X = np.array(input_sequences)
y = np.array(target_sequences)
print(f"input seq shape:{X.shape}")
print(f"target seq shape:{y.shape}")

input seq shape:(2554531, 100)
target seq shape:(2554531,)


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((X,y))
dataset = dataset.shuffle(buffer_size=10000).batch(128,drop_remainder=False)

In [ ]:
print(f"shape of  dataser:{X.shape} ,{y.shape}")

shape of  dataser:(2554531, 100) ,(2554531,)


In [ ]:
def create_model (vocab_size, embed_dim, rnn_neurons):
  model = tf.keras. Sequential([
    tf.keras.layers. Embedding (input_dim=vocab_size, output_dim=embed_dim),
    tf.keras.layers. LSTM (rnn_neurons),
    tf.keras.layers. Dense (vocab_size, activation='softmax')
    ])
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
embed_dim = 256
rnn_neurons=512
model = create_model(len(unique_chars), embed_dim,rnn_neurons)

In [ ]:
model.fit(dataset,epochs=5)

Epoch 1/20
19958/19958 ━━━━━━━━━━━━━━━━━━━━ 912s 45ms/step - accuracy: 0.5241 - loss: 1.5937
Epoch 2/20
19958/19958 ━━━━━━━━━━━━━━━━━━━━ 925s 46ms/step - accuracy: 0.6169 - loss: 1.2456
Epoch 3/20
19958/19958 ━━━━━━━━━━━━━━━━━━━━ 911s 46ms/step - accuracy: 0.6311 - loss: 1.1946
Epoch 4/20
19958/19958 ━━━━━━━━━━━━━━━━━━━━ 922s 46ms/step - accuracy: 0.6382 - loss: 1.1698
Epoch 5/20
19958/19958 ━━━━━━━━━━━━━━━━━━━━ 921s 46ms/step - accuracy: 0.6425 - loss: 1.1544
Epoch 6/20
19958/19958 ━━━━━━━━━━━━━━━━━━━━ 910s 46ms/step - accuracy: 0.6446 - loss: 1.1475
Epoch 7/20
14594/19958 ━━━━━━━━━━━━━━━━━━━━ 4:04 46ms/step - accuracy: 0.6448 - loss: 1.1452

In [ ]:
def generate_text(model, start_seed, gen_size=100, temp=1.0):

  num_generate = gen_size
  input_eval = [char_to_ind [s] for s in start_seed]
  input_eval = tf.expand_dims (input_eval, 0)
  text_generated = []
  for i in range (num_generate):

    predictions = model (input_eval)
    predictions = tf.squeeze (predictions, 0)
    predictions = predictions / temp
    predictions = tf.expand_dims (predictions, 0)
    predicted_id = tf.random.categorical (predictions, num_samples=1) [-1, 0].numpy()
    input_eval = tf.expand_dims ( [predicted_id], 0)
    text_generated.append(ind_to_char [predicted_id])
  return start_seed + ''.join(text_generated)


In [ ]:
print(generate_text(model , 'در دل شب' , gen_size=50))